In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../src')

In [3]:
import numpy as np
import pandas as pd
import glob
import json
import logging

In [4]:
DATASET_PATH = '../../datasets/amazon-sneakers'

In [9]:
products_detail_files         = !find {DATASET_PATH} -type f -name '??????????.json'
products_detail_variant_files = !find {DATASET_PATH} -type f -name '*_variant_*.json'

products_detail_files = products_detail_files + products_detail_variant_files
# products_detail_files

In [10]:
def load_json(path):
    with open(path,'r') as f:
        return json.load(f)

def to_product_detail_json(path):
    json = load_json(path)
    json['variant'] = ('variant' in path)
    return json
    
product_details = [to_product_detail_json(file) for file in products_detail_files]

In [11]:
def price(data):    
    if 'price_range' in data and data['price_range'] != None:
        price_range = data['price_range']
        
        if isinstance(price_range, str):
            return price_range.split('Details The List Price')[0].replace('List Price:', '').replace('$', '')
        elif 'from' in price_range and 'to' in price_range:
            return (price_range['to'] - price_range['from']) / 2
        elif 'from' in price_range:
            return price_range['from']
        elif 'to' in price_range:
            return price_range['to']

    return 0

def to_product_detail_row(data):
    try:        
        return {
            'id'         : data['id'],
            'title'      : data['title'],
            'description': data['description'],
            'variant'    : data['variant'],
            'url'        : data['images'][0][0],
            'rating'     : data['rating'],
            'reviews'    : data['reviews_count'],
            'price'      : price(data)
        }
    except Exception as e:
        logging.error(f'{e}. Data: {data}')
        

product_detail_rows = [to_product_detail_row(pd) for pd in product_details if pd['id']]

In [12]:
product_detail_rows[0]

{
    'id': 'B089PV3D1G',
    'title': "Superga Women's 2750 Cotu Classic Sneaker",
    'description': None,
    'variant': False,
    'url': 'https://m.media-amazon.com/images/I/612WcQ1+bIL._AC_UX695_.jpg',
    'rating': 4.2,
    'reviews': 7396,
    'price': 17.525
}

In [13]:
product_detail = pd.DataFrame(product_detail_rows)
product_detail

,id,title,description,variant,url,rating,reviews,price
0,B089PV3D1G,Superga Women's 2750 Cotu Classic Sneaker,None,False,https://m.media-amazon.com/images/I/612WcQ1+bI...,4.2,7396,17.525
1,B07MTVLHYS,New Suede Shoe Cleaner Kit. Suede Brush & Sued...,None,False,https://m.media-amazon.com/images/I/71tbpuN6ZI...,4.4,872,0
2,B0BGN5GY1G,New Balance Women's Dynasoft Nergize V3 Cross ...,None,False,https://m.media-amazon.com/images/I/71pR+J-gp4...,4.4,117,0.0
3,B085ZL5DW9,NCNDB Women's Camouflage Wedges Sneakers Inner...,welcome.our Sneaker shoes size tabs are chines...,False,https://m.media-amazon.com/images/I/61reE-na08...,4.2,191,1.82
4,B0785RGRWX,Reebok Men's Classic Harman Run Sneaker,None,False,https://m.media-amazon.com/images/I/71FaYc6pSM...,4.6,6501,10.465
...,...,...,...,...,...,...,...,...
1780,B07NGY3X4K,Saucony Women's VERSAFOAM Cohesion 12 Road Run...,None,True,https://m.media-amazon.com/images/I/81EyumbDg8...,4.5,9698,7.5
1781,B07KX68ZF8,PUMA Women's Carina Sneaker,None,True,https://m.media-amazon.com/images/I/71u2Muh5O5...,4.5,16647,6.78
1782,B08M41C7LJ,New Balance Women's 237 V1 Classic Sneaker,The running shoes of the 1970s might best be d...,True,https://m.media-amazon.com/images/I/51s0oWE4Hk...,4.3,331,75.5
1783,B0955T46NW,New Balance Women's Fresh Foam X70 V1 Sneaker,None,True,https://m.media-amazon.com/images/I/71i5pFo8hr...,4.6,1694,49.0


In [14]:
product_detail.price.value_counts()

0.000     437
46.885     55
45.640     32
3.500      27
54.000     23
         ... 
0.010       1
19.135      1
16.480      1
9.380       1
49.000      1
Name: price, Length: 453, dtype: int64

In [15]:
product_detail.to_json(f'{DATASET_PATH}/items.json', orient="records")

In [16]:
def to_reviews_json(path):
    reviews = load_json(path)
    for review in reviews:
        review['item_id']  = path.split('/')[-1].split('_')[0]
        review['username'] = review['user']
        del review['user']
    return reviews


product_reviews = []
for file in glob.glob(f'{DATASET_PATH}/*_reviews.json'):
    product_reviews.extend(to_reviews_json(file))

In [17]:
user_interactions = pd.DataFrame(product_reviews)
user_interactions = user_interactions[['username', 'item_id', 'rating', 'message']]

In [18]:
user_interactions.to_json(f'{DATASET_PATH}/interactions.json', orient="records")

# Database prepare

In [30]:
items      = pd.read_json(f'{DATASET_PATH}/items.json')
interactions = pd.read_json(f'{DATASET_PATH}/interactions.json')

In [31]:
items['description'] = items['description'] + ' (Price: ' + items['price'].astype(str) + ')'

In [32]:
def sequence_from(df, source, target):
    df[target] = pd.factorize(df[source])[0] + 1
    return df

select = lambda df, columns: df[columns]
rename = lambda df, columns: df.rename(columns=columns)


items = items \
    .pipe(sequence_from, source='id', target='item_id') \
    .pipe(select, ['item_id', 'id', 'title', 'description', 'url']) \
    .pipe(rename, { 'id': 'original_id', 'title': 'name', 'url': 'image'})

items.head(5)

,item_id,original_id,name,description,image
0,1,B089PV3D1G,Superga Women's 2750 Cotu Classic Sneaker,NaN,https://m.media-amazon.com/images/I/612WcQ1+bI...
1,2,B07MTVLHYS,New Suede Shoe Cleaner Kit. Suede Brush & Sued...,NaN,https://m.media-amazon.com/images/I/71tbpuN6ZI...
2,3,B0BGN5GY1G,New Balance Women's Dynasoft Nergize V3 Cross ...,NaN,https://m.media-amazon.com/images/I/71pR+J-gp4...
3,4,B085ZL5DW9,NCNDB Women's Camouflage Wedges Sneakers Inner...,welcome.our Sneaker shoes size tabs are chines...,https://m.media-amazon.com/images/I/61reE-na08...
4,5,B0785RGRWX,Reebok Men's Classic Harman Run Sneaker,NaN,https://m.media-amazon.com/images/I/71FaYc6pSM...


In [33]:
interactions = interactions \
    .pipe(sequence_from, source='username', target='user_id') \
    .pipe(select, ['item_id', 'rating', 'user_id']) \
    .pipe(rename, {'item_id': 'original_item_id'})

interactions.head(5)

,original_item_id,rating,user_id
0,B0BG8YCNZM,4.0,1
1,B0BG8YCNZM,4.0,2
2,B0BG8YCNZM,4.0,3
3,B0BG8YCNZM,5.0,4
4,B0BG8YCNZM,5.0,5


In [34]:
interactions = interactions.merge(items, left_on  = 'original_item_id', right_on = 'original_id') \
    .pipe(select, ['user_id', 'item_id', 'rating'])

interactions.head(5)

,user_id,item_id,rating
0,1,191,4.0
1,1,191,4.0
2,1,191,4.0
3,1,191,4.0
4,1,191,4.0


In [35]:
interactions.to_json(f'{DATASET_PATH}/db_interactions.json', orient="records")

In [36]:
items = items.pipe(select, ['item_id', 'name', 'description', 'image'])
items.head(5)

,item_id,name,description,image
0,1,Superga Women's 2750 Cotu Classic Sneaker,NaN,https://m.media-amazon.com/images/I/612WcQ1+bI...
1,2,New Suede Shoe Cleaner Kit. Suede Brush & Sued...,NaN,https://m.media-amazon.com/images/I/71tbpuN6ZI...
2,3,New Balance Women's Dynasoft Nergize V3 Cross ...,NaN,https://m.media-amazon.com/images/I/71pR+J-gp4...
3,4,NCNDB Women's Camouflage Wedges Sneakers Inner...,welcome.our Sneaker shoes size tabs are chines...,https://m.media-amazon.com/images/I/61reE-na08...
4,5,Reebok Men's Classic Harman Run Sneaker,NaN,https://m.media-amazon.com/images/I/71FaYc6pSM...


In [37]:
items.to_json(f'{DATASET_PATH}/db_items.json', orient="records")